## 1. Importing Libraries and Loading Data

In [3]:
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import resample
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.samplers import TPESampler
import joblib
import os
import shutil

# Load the data
df = pd.read_csv("../dibh_clinical_only/original_training_data.csv")
df_internal_val = pd.read_csv("../dibh_clinical_only/final_internal_validation13052024.csv")



## 2. Data Prepration

In [ ]:
# Prepare the data
data_day1 = df[df['day'] == 1]
X = data_day1.drop(['crnumber', 'day', 'DIBH_Y0N1'], axis=1)
y = data_day1['DIBH_Y0N1']
X_t, X_v, y_t, y_v = train_test_split(X, y, test_size=0.30, random_state=42)

data_day1_val = df_internal_val[df_internal_val['day'] == 1]
X_int_val = data_day1_val.drop(['crnumber', 'day', 'DIBH_Y0N1'], axis=1)
y_int_val = data_day1_val['DIBH_Y0N1']

## 3. Preprocessing Pipelines

In [4]:
# Define the feature categories
categorical_features = ['al_N0_Y1', 'surgery_BCS1MRM2', 'chemo_No0_Adj1_NAdj2', 'comorb_no0_cardio1_others2']
continuous_features = ['age', 'BMI', 'ul_amp', 'll_amp', 'average_amp', 'ahd']

# Create pipelines for numerical and categorical features
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, continuous_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

## 4. Hyperparameter Tuning with Optuna

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    gb_clf = make_pipeline(preprocessor, GradientBoostingClassifier(**params, random_state=42))
    cv_roc_auc = cross_val_score(gb_clf, X_t, y_t, cv=10, scoring='roc_auc').mean()
    return cv_roc_auc

# Create and optimize the study
storage_name = "sqlite:///db.sqlite3"
sampler = TPESampler(seed=72)
study = optuna.create_study(direction='maximize', sampler=sampler, storage=storage_name, study_name="GB_one_day_assessment_final04")
study.optimize(objective, n_trials=1000)

# Load the best parameters
loaded_study = optuna.create_study(study_name="GB_one_day_assessment_final01", storage=storage_name, load_if_exists=True)
best_params = loaded_study.best_params

## 5. Bootstrap Sampling and Model Training

In [ ]:
# Initialize lists to store results
top_models = []
fraction_of_positives_calibrated_list = []
mean_predicted_value_calibrated_list = []
fraction_of_positives_non_calibrated_list = []
mean_predicted_value_non_calibrated_list = []
results = []

# Perform bootstrap sampling and model training
for i in range(1000):
    X_resampled, y_resampled = resample(X_t, y_t)
    X_train_resampled, X_val_resampled, y_train_resampled, y_val_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2)

    gb_pipeline = make_pipeline(preprocessor, GradientBoostingClassifier(**best_params, random_state=42))
    gb_pipeline.fit(X_train_resampled, y_train_resampled)
    y_proba_non_calibrated = gb_pipeline.predict_proba(X_v)[:, 1]

    best_gb_pipeline = make_pipeline(preprocessor, GradientBoostingClassifier(**best_params, random_state=42))
    model = CalibratedClassifierCV(best_gb_pipeline, method='isotonic', cv=10)
    model.fit(X_train_resampled, y_train_resampled)
    y_proba_calibrated = model.predict_proba(X_v)[:, 1]

    fraction_of_positives_non_calibrated, mean_predicted_value_non_calibrated = calibration_curve(y_v, y_proba_non_calibrated, n_bins=6)
    if len(fraction_of_positives_non_calibrated) == 6:
        fraction_of_positives_non_calibrated_list.append(fraction_of_positives_non_calibrated)
        mean_predicted_value_non_calibrated_list.append(mean_predicted_value_non_calibrated)

    fraction_of_positives_calibrated, mean_predicted_value_calibrated = calibration_curve(y_v, y_proba_calibrated, n_bins=6)
    if len(fraction_of_positives_calibrated) == 6:
        fraction_of_positives_calibrated_list.append(fraction_of_positives_calibrated)
        mean_predicted_value_calibrated_list.append(mean_predicted_value_calibrated)

    fpr, tpr, thresholds = roc_curve(y_v, y_proba_calibrated)
    roc_auc = auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    y_pred_optimal = (y_proba_calibrated >= optimal_threshold).astype(int)

    model_info = {
        'model': model,
        'fpr': fpr,
        'tpr': tpr,
        'thresholds': thresholds,
        'roc_auc': roc_auc,
        'optimal_threshold': optimal_threshold,
        'accuracy': accuracy_score(y_v, y_pred_optimal),
        'precision': precision_score(y_v, y_pred_optimal),
        'recall': recall_score(y_v, y_pred_optimal),
        'f1_score': f1_score(y_v, y_pred_optimal),
        'confusion_matrix': confusion_matrix(y_v, y_pred_optimal),
        'classification_report': classification_report(y_v, y_pred_optimal)
    }

    model_info_without_model = {key: value for key, value in model_info.items() if key != 'model'}
    results.append(model_info_without_model)

    if len(top_models) < 10:
        top_models.append(model_info)
    else:
        min_index = min(range(len(top_models)), key=lambda x: (top_models[x]['roc_auc'], top_models[x]['recall']))
        if roc_auc > top_models[min_index]['roc_auc']:
            top_models[min_index] = model_info

    if (i + 1) % 25 == 0:
        print(f"Bootstrap sample no. {i + 1} ------ Finished")

## 6. Summary of Results

In [ ]:
# Summarize the results
df_results = pd.DataFrame(results)

metric_summary = df_results[['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']].agg(['mean', 'std', 'min', 'max'])
metric_summary.loc['95% CI lower'] = metric_summary.loc['mean'] - 1.96 * metric_summary.loc['std']
metric_summary.loc['95% CI upper'] = metric_summary.loc['mean'] + 1.96 * metric_summary.loc['std']

print(metric_summary)

## 7. Saving the Models

In [ ]:
# Save the top models and preprocessor
model_folder_path = '../saved_models/one_day_gb_top10'

if os.path.exists(model_folder_path):
    shutil.rmtree(model_folder_path)
os.makedirs(model_folder_path)

for i, m in enumerate(top_models):
    model_path = os.path.join(model_folder_path, f'top_model_{i+1}.joblib’)
joblib.dump(m[‘model’], model_path)

preprocessor_path = os.path.join(model_folder_path, ‘preprocessor.joblib’)
joblib.dump(preprocessor, preprocessor_path)

print(f”All top models have been saved to folder: {model_folder_path}”)
print(f”Preprocessor has been saved as {preprocessor_path}”)

## 8. Creating an Ensemble Classifier

In [ ]:
# Define the ensemble classifier
class ThresholdedEnsembleClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        return self
    
    def predict_proba(self, X):
        probabilities = np.mean([model.predict_proba(X)[:, 1] for model, _ in self.models], axis=0)
        return probabilities
    
    def predict(self, X):
        predictions = np.array([model.predict_proba(X)[:, 1] >= threshold for model, threshold in self.models]).astype(int).T
        final_predictions = mode(predictions, axis=1)[0].flatten()
        return final_predictions

# Save the ensemble classifier
ensemble_models = [(m['model'], m['optimal_threshold']) for m in top_models]
ensemble_classifier = ThresholdedEnsembleClassifier(ensemble_models)

joblib_file = '../saved_models/one_day_gb_top10/ensemble_classifier_gb.joblib'
joblib.dump(ensemble_classifier, joblib_file)

## 9. Evaluation on Internal Validation Data

In [ ]:
# Evaluate the ensemble classifier on internal validation data
ensemble_classifier = joblib.load(joblib_file)
y_pred = ensemble_classifier.predict(X_int_val)
y_proba = ensemble_classifier.predict_proba(X_int_val)

print("Ensemble Accuracy:", accuracy_score(y_int_val, y_pred))
print("Ensemble ROC AUC:", roc_auc_score(y_int_val, y_proba))
print("Ensemble Precision:", precision_score(y_int_val, y_pred))
print("Ensemble Recall:", recall_score(y_int_val, y_pred))
print("Ensemble F1 Score:", f1_score(y_int_val, y_pred))

# Calculate the confusion matrix
cm = confusion_matrix(y_int_val, y_pred)

# Define the class labels
class_labels = ['DIBH', 'NonDIBH']

# Create the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

# Save the figure
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# Print classification report
print(classification_report(y_int_val, y_pred, target_names=class_labels))